In [37]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [38]:
#Load the trained model, scaler, oohe
model = load_model('model.h5')

#load scaler and ohe
with open('label_encode_gender.pkl', 'rb') as file:
    label_encode_gender = pickle.load(file)

with open('ohe_geo.pkl', 'rb') as file:
    ohe_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [39]:
#Example ip data
input_data={
    'CreditScore':100,
    'Geography':'Germany',
    'Gender':'Female',
    'Age':25,
    'Tenure':10,	
    'Balance': 6000,
    'NumOfProducts': 4,
    'HasCrCard': 4,
    'IsActiveMember': 1,
    'EstimatedSalary': 5000
}

In [40]:
geo_encoded = ohe_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

v:\GenAI\ANNclassification\annvenv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [41]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,100,Germany,Female,25,10,6000,4,4,1,5000


In [42]:
input_df['Gender'] = label_encode_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,100,Germany,0,25,10,6000,4,4,1,5000


In [43]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,100,Germany,0,25,10,6000,4,4,1,5000


In [44]:
#Concat with OHE
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,100,0,25,10,6000,4,4,1,5000,0.0,1.0,0.0


In [45]:
input_scaled = scaler.transform(input_df)

In [46]:
input_scaled

array([[-5.72485068, -1.09499335, -1.32129295,  1.7337772 , -1.12240462,
         4.25868381,  7.21786376,  0.97481699, -1.65923237, -0.99850112,
         1.72572313, -0.57638802]])

In [47]:
#Predict churn

prediction = model.predict(input_scaled)
prediction_proba = prediction[0][0]
prediction_proba

1/1 [==============================] - 0s 163ms/step


1.4425408

In [48]:
if prediction_proba>0.5:
    print("Customer is likely to churn")
else:
    print("Customer is likely to stay")

Customer is likely to churn
